In [46]:
import os
import pandas as pd
from datetime import timedelta

data = pd.read_csv("../../data/supermarket_sales - Sheet1.csv")
print(data)

data = data[["Branch", "Product line", "Date", "Quantity"]]
data["Date"] = pd.to_datetime(data["Date"])
print(data)

min_date = data["Date"].min()
max_date = data["Date"].max()
delta = max_date - min_date

for i in range(delta.days + 1):
    day = min_date + timedelta(days=i)
    for branch in pd.unique(data["Branch"]):
        for product_line in pd.unique(data["Product line"]):
            data.loc[len(data)] = [branch, product_line, day, 0]
data = data.groupby(["Branch", "Product line", "Date"])["Quantity"].sum().reset_index()
print(data)
data.to_pickle("model_data.pkl")

      Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyle       46.33         7  16.2155  

In [20]:
import os
import pandas as pd

data = pd.read_pickle("./model_data.pkl")
print(data)

train_data = pd.DataFrame(columns=["Branch", "Product line", "Month"] + [f"{i}" for i in range(14)] + ["Class"])
branches = pd.unique(data["Branch"])
product_lines = pd.unique(data["Product line"])

for branch in branches:
    for product_line in product_lines:
        cur_frame = data[(data["Branch"] == branch) & (data["Product line"] == product_line)]
        values = data["Quantity"]
        cur_slide = values.iloc[:14].to_list()
        for i in range(14, values.shape[0]):
            label = 1
            
            if cur_slide[-1] - values[i] >= cur_slide[-1] * 0.33:
                label = 0
            elif cur_slide[-1] - values[i] <= cur_slide[-1] * -0.33:
                label = 2
            train_data.loc[len(train_data.index)] = [branch, product_line, data.iloc[i]["Date"].month] + cur_slide + [label]
            
            cur_slide.pop(0)
            cur_slide.append(values[i])
            
train_data = pd.concat([train_data, pd.get_dummies(train_data["Branch"])], axis=1)
train_data = pd.concat([train_data, pd.get_dummies(train_data["Product line"])], axis=1)

train_data.pop("Branch")
train_data.pop("Product line")
train_data = train_data[[c for c in train_data if c not in [f"{i}" for i in range(14)]] 
       + [f"{i}" for i in range(14)]]
            
print(train_data)
train_data.to_pickle("transformed_model_data.pkl")

     Branch            Product line       Date  Quantity
0         A  Electronic accessories 2019-01-01        10
1         A  Electronic accessories 2019-01-02         0
2         A  Electronic accessories 2019-01-03         0
3         A  Electronic accessories 2019-01-04         0
4         A  Electronic accessories 2019-01-05         7
...     ...                     ...        ...       ...
1597      C       Sports and travel 2019-03-26         0
1598      C       Sports and travel 2019-03-27         0
1599      C       Sports and travel 2019-03-28         0
1600      C       Sports and travel 2019-03-29         0
1601      C       Sports and travel 2019-03-30         0

[1602 rows x 4 columns]
       Month  Class  A  B  C  Electronic accessories  Fashion accessories  \
0          1      2  1  0  0                       1                    0   
1          1      0  1  0  0                       1                    0   
2          1      0  1  0  0                       1        

In [6]:
import os
import pandas as pd

data = pd.read_pickle("./transformed_model_data.pkl")
#print(data)
train_data = data.sample(frac=0.7)
valid_data = data.drop(train_data.index)
test_data = valid_data.sample(frac=0.5)
valid_data = valid_data.drop(test_data.index)

train_0 = train_data[train_data["Class"] == 0]
train_1 = train_data[train_data["Class"] == 1]
train_2 = train_data[train_data["Class"] == 2]
max_train = (train_0 if train_0.shape[0] >= train_1.shape[0] and train_0.shape[0] >= train_2.shape[0] else
             train_1 if train_1.shape[0] >= train_0.shape[0] and train_1.shape[0] >= train_2.shape[0] else
             train_2)

valid_0 = valid_data[valid_data["Class"] == 0]
valid_1 = valid_data[valid_data["Class"] == 1]
valid_2 = valid_data[valid_data["Class"] == 2]
max_valid = (valid_0 if valid_0.shape[0] >= valid_1.shape[0] and valid_0.shape[0] >= valid_2.shape[0] else
             valid_1 if valid_1.shape[0] >= valid_0.shape[0] and valid_1.shape[0] >= valid_2.shape[0] else
             valid_2)

for data_item in [train_0, train_1, train_2]:
    cur_num = data_item.shape[0]
    while cur_num + data_item.shape[0] < max_train.shape[0]:
        train_data = pd.concat([train_data, data_item])
        cur_num += data_item.shape[0]

for data_item in [valid_0, valid_1, valid_2]:
    cur_num = data_item.shape[0]
    while cur_num + data_item.shape[0] < max_valid.shape[0]:
        valid_data = pd.concat([valid_data, data_item])
        cur_num += data_item.shape[0]
        
print((data["Class"] == 0).sum() / data.shape[0])
print((data["Class"] == 1).sum() / data.shape[0])
print((data["Class"] == 2).sum() / data.shape[0])
print()

print((train_data["Class"] == 0).sum() / train_data.shape[0])
print((train_data["Class"] == 1).sum() / train_data.shape[0])
print((train_data["Class"] == 2).sum() / train_data.shape[0])
print()

print((valid_data["Class"] == 0).sum() / valid_data.shape[0])
print((valid_data["Class"] == 1).sum() / valid_data.shape[0])
print((valid_data["Class"] == 2).sum() / valid_data.shape[0])
print()

print(pd.merge(train_data, valid_data).shape[0])
print()

train_data.to_pickle("train_data.pkl")
valid_data.to_pickle("valid_data.pkl")
test_data.to_pickle("test_data.pkl")

0.6032745591939547
0.0535264483627204
0.3431989924433249

0.39043164766637434
0.3851375491246874
0.22443080320893827

0.4017159491343649
0.37750880956028804
0.220775241305347

0



In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(seed=42)

BATCH_SIZE = 256

train_data_pd = pd.read_pickle("./train_data.pkl")
valid_data_pd = pd.read_pickle("./valid_data.pkl")
train_class_pd = train_data_pd.pop("Class")
valid_class_pd = valid_data_pd.pop("Class")

input_data = tf.convert_to_tensor(train_data_pd.values.reshape(-1, 1, train_data_pd.shape[1]))
input_labels = tf.convert_to_tensor(train_class_pd.values.reshape(-1, 1, 1), dtype=tf.float32)
valid_data = tf.convert_to_tensor(valid_data_pd.values.reshape(-1, 1, valid_data_pd.shape[1]))
valid_labels = tf.convert_to_tensor(valid_class_pd.values.reshape(-1, 1, 1), dtype=tf.float32)

model = keras.models.Sequential([
    layers.LSTM(units=30, input_shape=(1, train_data_pd.shape[1]), return_sequences=True),
    layers.Dense(units=1),
    layers.Reshape([1,-1])
])
loss = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.RMSprop(learning_rate=0.0025)
metrics = [keras.metrics.CategoricalCrossentropy(), keras.metrics.MeanSquaredError()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)
model.fit(x=input_data, y=input_labels, epochs=50, batch_size=BATCH_SIZE)
print("Valid Accuracy: ", float(tf.divide(tf.reduce_sum(tf.cast(tf.equal(tf.round(model(input_data)), input_labels), tf.int32)), tf.shape(input_labels)[0])))
print("Valid Accuracy: ", float(tf.divide(tf.reduce_sum(tf.cast(tf.equal(tf.round(model(valid_data)), valid_labels), tf.int32)), tf.shape(valid_labels)[0])))

model.save("./model.h5")

Epoch 1/50
121/121 [==============================] - 2s 2ms/step - loss: 0.5566 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.5566
Epoch 2/50
121/121 [==============================] - 0s 2ms/step - loss: 0.4274 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.4274
Epoch 3/50
121/121 [==============================] - 0s 2ms/step - loss: 0.3559 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.3559
Epoch 4/50
121/121 [==============================] - 0s 2ms/step - loss: 0.2978 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.2978
Epoch 5/50
121/121 [==============================] - 0s 2ms/step - loss: 0.2531 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.2531
Epoch 6/50
121/121 [==============================] - 0s 2ms/step - loss: 0.2194 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.2194
Epoch 7/50
121/121 [==============================] - 0s 2ms/step - loss: 0.1942 - categorical_crossentrop

In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(seed=42)

BATCH_SIZE = 256

train_data_pd = pd.read_pickle("./train_data.pkl")
valid_data_pd = pd.read_pickle("./valid_data.pkl")
train_class_pd = train_data_pd.pop("Class")
valid_class_pd = valid_data_pd.pop("Class")

input_data = tf.convert_to_tensor(train_data_pd.values.reshape(-1, 1, train_data_pd.shape[1]))
input_labels = tf.convert_to_tensor(train_class_pd.values.reshape(-1, 1, 1), dtype=tf.float32)
valid_data = tf.convert_to_tensor(valid_data_pd.values.reshape(-1, 1, valid_data_pd.shape[1]))
valid_labels = tf.convert_to_tensor(valid_class_pd.values.reshape(-1, 1, 1), dtype=tf.float32)

model = keras.models.load_model("./model.h5")
model.fit(x=input_data, y=input_labels, epochs=10, batch_size=BATCH_SIZE)

print("Valid Accuracy: ", float(tf.divide(tf.reduce_sum(tf.cast(tf.equal(tf.round(model(input_data)), input_labels), tf.int32)), tf.shape(input_labels)[0])))
print("Valid Accuracy: ", float(tf.divide(tf.reduce_sum(tf.cast(tf.equal(tf.round(model(valid_data)), valid_labels), tf.int32)), tf.shape(valid_labels)[0])))

model.save("./model.h5")

Epoch 1/10
121/121 [==============================] - 1s 2ms/step - loss: 0.0286 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0286
Epoch 2/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0277 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0277
Epoch 3/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0273 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0273
Epoch 4/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0270 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0270
Epoch 5/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0264 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0264
Epoch 6/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0263 - categorical_crossentropy: 9.9420e-08 - mean_squared_error: 0.0263
Epoch 7/10
121/121 [==============================] - 0s 2ms/step - loss: 0.0257 - categorical_crossentrop

In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

test_data_pd = pd.read_pickle("./test_data.pkl")
test_class_pd = test_data_pd.pop("Class")

test_data = tf.convert_to_tensor(test_data_pd.values.reshape(-1, 1, test_data_pd.shape[1]))
test_labels = tf.convert_to_tensor(test_class_pd.values.reshape(-1, 1, 1), dtype=tf.float32)

model = keras.models.load_model("./model.h5")

predictions = tf.round(model(test_data))

equals_0 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(predictions, test_labels), tf.equal(test_labels, 0)), tf.int32)))
missed_0 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(test_labels, 0), tf.not_equal(predictions, 0)), tf.int32)))
guessed_0 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.not_equal(test_labels, 0), tf.equal(predictions, 0)), tf.int32)))

equals_1 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(predictions, test_labels), tf.equal(test_labels, 1)), tf.int32)))
missed_1 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(test_labels, 1), tf.not_equal(predictions, 1)), tf.int32)))
guessed_1 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.not_equal(test_labels, 1), tf.equal(predictions, 1)), tf.int32)))

equals_2 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(predictions, test_labels), tf.equal(test_labels, 2)), tf.int32)))
missed_2 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(test_labels, 2), tf.not_equal(predictions, 2)), tf.int32)))
guessed_2 = float(tf.reduce_sum(tf.cast(tf.logical_and(tf.not_equal(test_labels, 2), tf.equal(predictions, 2)), tf.int32)))

print("Precision 0:", equals_0 / (equals_0 + guessed_0))
print("Recall 0:", equals_0 / (equals_0 + missed_0))
print("Precision 1:", equals_1 / (equals_1 + guessed_1))
print("Recall 1:", equals_1 / (equals_1 + missed_1))
print("Precision 2:", equals_2 / (equals_2 + guessed_2))
print("Recall 2:", equals_2 / (equals_2 + missed_2))
print("Accuracy: ", float(tf.divide(tf.reduce_sum(tf.cast(tf.equal(predictions, test_labels), tf.int32)), tf.shape(test_labels)[0])))

Precision 0: 1.0
Recall 0: 0.9657823913879278
Precision 1: 0.7058823529411765
Recall 1: 1.0
Precision 2: 1.0
Recall 2: 0.9814941740918437
Accuracy:  0.9729477611940298
